In [8]:
import json
import requests
import threading
import re
import traceback

URL = 'http://3.84.244.86:8080/enhancer/chain/PurchaseOrder'
FLATTEN_URL = 'http://34.80.26.185:8086/PO_Processing/GetDataSolr.flatteringTemp'
CHOOSE_URL = 'http://34.80.26.185:8086/PO_Processing/GetDataSolr.Temp'
VALIDATION_URL = 'http://34.80.26.185:8086/PO_Processing/ReArrangingofData'
REGEX_URL = 'http://35.221.160.146:5020/regexResult'
OVERLAP_URL = 'http://35.186.166.22:8082/portal/servlet/service/Poheader.poi'

LABEL = 'http://fise.iks-project.eu/ontology/entity-reference'
TEXT_LABEL = 'http://fise.iks-project.eu/ontology/selected-text'
RELATION_LABEL = 'http://purl.org/dc/terms/relation'
SELECTED_LABEL = 'http://bizlem.io/PurchaseOrderProcessing#'
SITE_LABEL = 'http://stanbol.apache.org/ontology/entityhub/entityhub#site'

In [3]:
def middle(value, left, right):
    return value >= left and value <= right

def get_x(obj, index=0):
    return obj['bounding_poly']['vertices'][index]['x']

def get_y(obj, index=0):
    return obj['bounding_poly']['vertices'][index]['y']

def to_float(s):
    s_format = ''.join(re.split('[^\d\.]', s))
    return float(s_format)

def is_float(s):
    s_format = ''.join(re.split(',', s))
    try:
        num = float(s_format)
        return True
    except ValueError:
        return False

In [4]:
folder = 'c57'
path = '../data/p4materials/' + folder + '.json'
# path = '/home/pexea12/N2000HZ118-19-0149628.json'
with open(path) as f:
    data = json.load(f)
    data_sum = data[0]
    data = data[1:]

In [5]:
def parseJSON(data, x_thres, y_thres):
    response = []
    
    # Build histogram
    y = [ (get_y(obj) + get_y(obj, 2)) // 2 for obj in data ]
    y.sort()

    cur_y = y[0]
    hist_y = { cur_y: cur_y }

    for yc in y:
        if yc == cur_y: 
            continue
        if yc - cur_y > y_thres:
            cur_y = yc
        hist_y[yc] = cur_y

    data.sort(key=lambda obj: hist_y[(get_y(obj) + get_y(obj, 2)) // 2] * 1000000 + get_x(obj))
    

    curLineYCoord = (get_y(data[0]) + get_y(data[0], 2)) // 2
    curXEnd = get_x(data[0], 2)
    curXStart = get_x(data[0])
    line = {
        'y': curLineYCoord,
        'words': [],
    }

    completedWord = data[0]['description']
    characters = set(['+', '-', ';', '-', '\\', "'", '"', '/'])
    open_chars = set(['(', '[', '{'])
    close_chars = set([')', ']', '}'])

    for i in range(1, len(data)):
        curWord = data[i]['description']
        yMid = (get_y(data[i]) + get_y(data[i], 2)) // 2
        xStart = get_x(data[i])


        if hist_y[yMid] <= hist_y[curLineYCoord] + y_thres and hist_y[yMid] >= hist_y[curLineYCoord] - y_thres:
            if xStart <= curXEnd + x_thres \
                or (xStart <= curXEnd + 2 * x_thres and ( \
                    curWord[0] in characters \
                    or curWord[0] in close_chars \
                    or ((curWord[0] in set([',', '.', '%'])) \
                        and '0' <= completedWord[-1] and completedWord[-1] <= '9') \
                    or completedWord[-1] in characters \
                    or completedWord[-1] in open_chars \
                    or (len(completedWord) >= 2 and completedWord[-1] in set([',', '.']) \
                        and '0' <= curWord[0] and curWord[0] <= '9' \
                        and '0' <= completedWord[-2] and completedWord[-2] <= '9'))):
                    completedWord += curWord
                    curXEnd = get_x(data[i], 2)
            else:
                line['words'].append({
                    'word': completedWord,
                    'x1': curXStart,
                    'x2': curXEnd,
                })

                curXEnd = get_x(data[i], 2)
                completedWord = data[i]['description']
                curXStart = get_x(data[i])
        else:
            line['words'].append({
                'word': completedWord,
                'x1': curXStart,
                'x2': curXEnd,
            })

            response.append(line)
            curLineYCoord = (get_y(data[i]) + get_y(data[i], 2)) // 2
            curXStart = get_x(data[i])
            curXEnd = get_x(data[i], 2)

            completedWord = data[i]['description']
            line = {
                'y': curLineYCoord,
                'words': [],
            }

    return response

In [6]:
slist = parseJSON(data, 5, 10)

for line_index, line in enumerate(slist):
    s = ' '.join([ word['word'] for word in line['words'] ])
    
    print(line_index, line['y'], s)
    
max_y = slist[-1]['y']
max_x = 0
for line in slist:
    if len(line['words']) == 0:
        continue
        
    last_word = line['words'][-1]['x2']
    if last_word > max_x:
        max_x = last_word
        
print('max', max_x, max_y)

0 154 Tel.:+91-22-4004 3516 / 17 /18
1 204 GSTIN : 27AAGFH1871A1ZQ M : 98211 19606 :hkinternational13 @ hotmail.com , info @ hkifasteners.com
2 327 ( HKI ) H . Ko international
3 449 MFG . & EXPORTERS OF : FASTENERS, INDUSTRIAL GASKETS & INDUSTRIAL HARDWARE.
4 506 SALES OFF. : 2, NEW BHAGWAN BHUVAN , GR . FLR ., 196 / 198, SAMUEL STREET,MASJID (WEST), MUMBAI- 9.
5 552 REGD .OFF.: C - 101,DWARKA C . H . S., BEHIND JAIN TEMPLE, DAULAT NAGAR , BORIVALI (EAST),MUMBAI-66.
6 628 TAX INVOICE (ORIGINAL FOR RECIPIENT)
7 631 .
8 651 100274
9 727 Invoice No. Dated
10 742 H .K .INTERNATIONAL
11 785 2, NEW BHAGWAN BHUVAN , GR . FLOOR , 788 23-Feb-2019
12 829 196 /198 , SAMUEL STREET, MASJID (WEST), Delivery Note Mode/Terms of Payment
13 872 MUMBAI -400009 788
14 884 1 Days
15 911 GSTIN /UIN : 27AAGFH1871A1ZQ
16 931 Supplier's Ref. Other Reference(s)
17 957 State Name: Maharashtra, Code : 27
18 986 788
19 1007 Buyer
20 1034 Buyer's Order No. Dated
21 1059 ESSAR STEEL INDIA LIMITED
22 1088 ESTL-DTAIG

In [7]:
for i, line in enumerate(slist):
    print(i, line['y'], [ word['word'] for word in line['words'] ])

0 154 ['Tel.:+91-22-4004', '3516', '/', '17', '/18']
1 204 ['GSTIN', ':', '27AAGFH1871A1ZQ', 'M', ':', '98211', '19606', ':hkinternational13', '@', 'hotmail.com', ',', 'info', '@', 'hkifasteners.com']
2 327 ['(', 'HKI', ')', 'H', '.', 'Ko', 'international']
3 449 ['MFG', '.', '&', 'EXPORTERS', 'OF', ':', 'FASTENERS,', 'INDUSTRIAL', 'GASKETS', '&', 'INDUSTRIAL', 'HARDWARE.']
4 506 ['SALES', 'OFF.', ':', '2,', 'NEW', 'BHAGWAN', 'BHUVAN', ',', 'GR', '.', 'FLR', '.,', '196', '/', '198,', 'SAMUEL', 'STREET,MASJID', '(WEST),', 'MUMBAI-', '9.']
5 552 ['REGD', '.OFF.:', 'C', '-', '101,DWARKA', 'C', '.', 'H', '.', 'S.,', 'BEHIND', 'JAIN', 'TEMPLE,', 'DAULAT', 'NAGAR', ',', 'BORIVALI', '(EAST),MUMBAI-66.']
6 628 ['TAX', 'INVOICE', '(ORIGINAL', 'FOR', 'RECIPIENT)']
7 631 ['.']
8 651 ['100274']
9 727 ['Invoice', 'No.', 'Dated']
10 742 ['H', '.K', '.INTERNATIONAL']
11 785 ['2,', 'NEW', 'BHAGWAN', 'BHUVAN', ',', 'GR', '.', 'FLOOR', ',', '788', '23-Feb-2019']
12 829 ['196', '/198', ',', 'SAMUEL', 'ST

In [11]:
sres = []
sres_words = []

for line in slist:
    sres.append({
        'y': line['y'],
        'words': [ word['word'] for word in line['words'] ],
    })
    
    sres_words.append({
        'y': line['y'],
        'words': [ word['word'] for word in line['words'] ]
    })
    

def request_line_and_replace(slist, sres, sres_words, line_index):
    sentence = ' '.join([ word for word in sres[line_index]['words'] ])
    res = {}
    try: 
        r = requests.post(URL, data=sentence.encode('utf-8'), headers={'Content-Type': 'application/pdf'})
        r = r.json()

        for obj in r:
            if LABEL in obj and RELATION_LABEL in obj:
                res[obj[LABEL][0]['@id']] = obj[RELATION_LABEL][0]['@id']

        line_type = None
        for obj in r:
            if TEXT_LABEL in obj and '@id' in obj:
                obj_id = obj['@id']
                for kurl, vid in res.items():
                    if vid == obj_id:
                        res[kurl] = obj[TEXT_LABEL][0]['@value']
                        
            if SITE_LABEL in obj:
                line_type = obj[SITE_LABEL][0]['@value']
       
        merging_r = {}
        words = slist[line_index]['words']
        
        word_index = 0
        while word_index < len(words):
            word = words[word_index]
            found = False
            
            for kurl, vtext in res.items():
                if kurl in merging_r:
                    continue
                    
                if found:
                    break
                    
                vtext_first_word = vtext.split(' ')[0]
                
                if vtext_first_word not in word['word']:
                    continue
                
                current_word_indexes = [ word_index ]
                current_s = word['word']
                current_i = word_index + 1
                while len(current_s) < len(vtext) and current_i < len(words):
                    current_word_indexes.append(current_i)
                    current_s += ' ' + words[current_i]['word']
                    current_i += 1
                
                if vtext not in current_s:
                    continue
                
                found = True
                merging_r[kurl] = current_word_indexes
                
            if not found:
                word_index += 1
        
#         print(merging_r)
#         print(res)
#         return

        removed_words = set()
        for url, merging_arr in merging_r.items():
            min_index = merging_arr[0]
            start_word = slist[line_index]['words'][min_index]
            sres_words[line_index]['words'][min_index] = url
            
            for word_index in merging_arr[1:]:
                removed_words.add(word_index)
                current_word = slist[line_index]['words'][word_index]
                start_word['word'] += ' ' + current_word['word']
                start_word['x2'] = current_word['x2']
        
        slist_newarr = []
        sres_words_newarr = []
        
        for word_index, word in enumerate(slist[line_index]['words']):
            if word_index in removed_words: continue
            
            slist_newarr.append(word)
            sres_words_newarr.append(sres_words[line_index]['words'][word_index])
            
        slist[line_index]['words'] = slist_newarr
        slist[line_index]['type'] = line_type
        sres_words[line_index]['words'] = sres_words_newarr
        
        sres[line_index]['words'] = list(res.keys())
#         print(sres_words_newarr)
                
    except Exception as e:
        traceback.print_exc()
                    
        
def request_all(sres):
    threads = [ threading.Thread(target=request_line_and_replace, args=(slist, sres, sres_words, line_index)) for line_index in range(len(sres)) ]
    for thread in threads:
        thread.start()
        
    for thread in threads:
        thread.join()

print('requesting...')
request_all(sres)
# request_line_and_replace(slist, sres, sres_words, 1)
print('finish requesting...')

# print(slist[1]['words'])
# print(sres_words[1]['words'])
for line_index, line in enumerate(sres):
    print(line_index, line['y'])
    print('\t', line['words'], slist[line_index]['type'])
    print('\t', [ word['word'] for word in slist[line_index]['words'] ])
    print('\t', sres_words[line_index]['words'])

requesting...
finish requesting...
0 154
	 [] None
	 ['Tel.:+91-22-4004', '3516', '/', '17', '/18']
	 ['Tel.:+91-22-4004', '3516', '/', '17', '/18']
1 204
	 [] None
	 ['GSTIN', ':', '27AAGFH1871A1ZQ', 'M', ':', '98211', '19606', ':hkinternational13', '@', 'hotmail.com', ',', 'info', '@', 'hkifasteners.com']
	 ['GSTIN', ':', '27AAGFH1871A1ZQ', 'M', ':', '98211', '19606', ':hkinternational13', '@', 'hotmail.com', ',', 'info', '@', 'hkifasteners.com']
2 327
	 [] None
	 ['(', 'HKI', ')', 'H', '.', 'Ko', 'international']
	 ['(', 'HKI', ')', 'H', '.', 'Ko', 'international']
3 449
	 [] None
	 ['MFG', '.', '&', 'EXPORTERS', 'OF', ':', 'FASTENERS,', 'INDUSTRIAL', 'GASKETS', '&', 'INDUSTRIAL', 'HARDWARE.']
	 ['MFG', '.', '&', 'EXPORTERS', 'OF', ':', 'FASTENERS,', 'INDUSTRIAL', 'GASKETS', '&', 'INDUSTRIAL', 'HARDWARE.']
4 506
	 [] None
	 ['SALES', 'OFF.', ':', '2,', 'NEW', 'BHAGWAN', 'BHUVAN', ',', 'GR', '.', 'FLR', '.,', '196', '/', '198,', 'SAMUEL', 'STREET,MASJID', '(WEST),', 'MUMBAI-', '9.']


In [8]:
for line_index, line in enumerate(sres_words):
    for word in line['words']:
        if SELECTED_LABEL in word:
            print(line_index, line['y'], word)

4 360 http://bizlem.io/PurchaseOrderProcessing#GSTIN
5 361 http://bizlem.io/PurchaseOrderProcessing#TaxInvoice
8 450 http://bizlem.io/PurchaseOrderProcessing#Purchase_Order_Number
8 450 http://bizlem.io/PurchaseOrderProcessing#Transport
10 556 http://bizlem.io/PurchaseOrderProcessing#Date
18 982 http://bizlem.io/PurchaseOrderProcessing#GSTIN
20 1127 http://bizlem.io/PurchaseOrderProcessing#CGST
20 1127 http://bizlem.io/PurchaseOrderProcessing#SGST
21 1141 http://bizlem.io/PurchaseOrderProcessing#DescriptionOfServices
22 1152 http://bizlem.io/PurchaseOrderProcessing#DescriptionOfServices
22 1152 http://bizlem.io/PurchaseOrderProcessing#Quantity
22 1152 http://bizlem.io/PurchaseOrderProcessing#Unit
22 1152 http://bizlem.io/PurchaseOrderProcessing#Discount
22 1152 http://bizlem.io/PurchaseOrderProcessing#GrossTotal
25 1206 http://bizlem.io/PurchaseOrderProcessing#Rate
33 1560 http://bizlem.io/PurchaseOrderProcessing#Tax_Code
45 2488 http://bizlem.io/PurchaseOrderProcessing#CGST
47 2544 ht

In [9]:
def concat(slist, line_index, index, n_lines=2):
    paragraph = ' '.join([ word['word'] for word in slist[line_index]['words'][index:] ]) + '\n'
    current_word = slist[line_index]['words'][index]
    
    for line in slist[line_index + 1:(line_index + 1) + n_lines]:
        for word in line['words']:
            if word['x1'] >= current_word['x1'] - 15:
                paragraph += word['word'] + ' '
        paragraph += '\n'
    
    return paragraph

def strip(s):
    return ''.join(re.split('[^a-zA-Z0-9]', s.lower()))


date_regex = '\d{1,2}\s?[\.\-\/\:]\s?\d{1,2}\s?[\.\-\/\:]\s?\d{4}|\d{1,2}[\s\-]+[A-Za-z]{3,}[\s\-]+\d{4}|\d{1,2}\s?\/\s?\d{1,2}\s?\/\s?\d{2}'
# (MM.DD.YYYY) | MM - DDD - YYYY | MM - DD - YYYY | MM / DD / YY
number_regex = '(?<![\.\d])(?:\,? ?\d+ ?)+(?:\. ?\d+)?(?!\d{0,} ?%)'
percentage_regex = '\d+\s?%|\d+\s?\.\s?%'

# processing text
patterns = {                                               
    'PAN_Number': [{                                               
        'regex': '[A-Z]{5}[0-9]{4}[A-Z]{1}',
        'main_key': 'PAN_Number',
    }],
    'TAN': [{                                               
        'regex': '[A-Z]{4}[0-9]{5}[A-Z]{1}',
        'main_key': 'TAN',
    }],
    'Purchase_Order_Number': [{                                   
        'regex': '\d{1,}',
        'main_key': 'Purchase_Order_Number',
    }],
    'GSTIN': [{
        'regex': '[A-Z0-9]{15}', 
        'main_key': 'GSTIN',
    }],
    'Invoice_Number': [{
        'regex': '\d{2,}|[A-Z]+\d+[A-Z]\s?(\d{2})?\s?\-\s?\d+|[A-Z]\s?\-?\s?\d+|[A-Z]+\/\d+\-\d+\/\d+(\/\d+)?',
        'main_key': 'Invoice_Number',
    }],
    # 12345 | SA18Y 09 - 00205 | S - 168 | GST/18-19/025
    'Invoice_Date': [{
        'regex': date_regex,
        'main_key': 'Invoice_Date',
    }],
    'Date': [{
        'regex': date_regex,
        'main_key': 'Invoice_Date',
    }],
    'L.R._No': [{
        'regex': '\d{1,2}\s?[(th)\-\.]?\s?([A-Za-z]{3,})\s?[\-\.]?\s?\d{4}|\d{1,2}\s?\.\s?\d{1,2}\s?\.\s?\d{2}|BY MONTH\s?\/?\s?\d{4}\s?\-\s?\d{4}',
        'main_key': 'L.R._No',
    }],
    'L.R._Dt': {
        'regex': date_regex,
        'main_key': 'L.R._Dt',
    },
    'GrossTotal': [{
        'regex': number_regex,
        'main_key': 'GrossTotal',
    }],
}


patterns_reversed = {
    'SGST': [
        {
            'regex': number_regex,
            'main_key': 'SGST_Amount',
        },
        {
            'regex': percentage_regex,
            'main_key': 'SGSTPercentage',
        },
    ],
    'CGST': [
        {
            'regex': number_regex,
            'main_key': 'CGST_Amount',
        },
        {
            'regex': percentage_regex,
            'main_key': 'CSGTPercentage',
        },
    ],
    'IGST': [
        {
            'regex': number_regex,
            'main_key': 'IGST_Amount',
        },
        {
            'regex': percentage_regex,
            'main_key': 'IGSTPercentage',
        },
    ],
    'SGSTPercentage': [{
        'regex': percentage_regex,
        'main_key': 'SGSTPercentage'
    }],
    'CGSTPercentage': [{
        'regex': percentage_regex,
        'main_key': 'CGSTPercentage'
    }],
    'IGSTPercentage': [{
        'regex': percentage_regex,
        'main_key': 'IGSTPercentage'
    }],
}

In [10]:
header_keys = set([
    'SNo',
    'DescriptionOfServices',
    'HSN/SAC',
    'Unit',
    'Quantity',
    'Rate',
    'AmountNumbers',
    'SGST',
    'CGST',
    'IGST',
])


selected = []
header_map_arr = []


end = False

for line_index, words in enumerate(slist):
    if len(sres[line_index]['words']) == 0: continue
        
    for word_index, word in enumerate(words['words']):
        url = sres_words[line_index]['words'][word_index]
        if SELECTED_LABEL not in url: continue
            
        url_key = url.split('#')[-1]
        if url_key not in header_keys: continue
            
        x1, x2 = word['x1'], word['x2']
        
        header_map_arr.append((
            SELECTED_LABEL + url_key,
            line_index,
            words['y'],
            x1,
            x2,
        ))

freq = [0] * max_y

for (url_key, _, y, _, _) in header_map_arr:
    start_y = max(0, y - 30)
    stop_y = min(max_y, y + 30)
    for i in range(start_y, stop_y + 1):
        freq[i] += 1

clusters_top = []
for i in range(1, max_y - 1):
    if freq[i] > freq[i - 1] and freq[i] > freq[i + 1]:
        clusters_top.append((i, freq[i]))
    elif freq[i] > freq[i - 1] and freq[i] == freq[i + 1]:
        for j in range(i + 1, max_y - 1):
            if freq[j] < freq[i]:
                clusters_top.append((i, freq[i]))
                break
            elif freq[j] > freq[i]:
                break

clusters_top.sort(key=lambda x: -x[1])
print(clusters_top)

cluster_index = 0
description_index = None
amount_index = None
cluster_headers = []
max_cluster_headers = []
amount_cluster_headers = []

while cluster_index < len(clusters_top): 
    current_y = clusters_top[cluster_index][0]
    for (url_key, line_index, y, x1, x2) in header_map_arr:
        if y >= current_y - 30 and y <= current_y + 30:
            cluster_headers.append((url_key, line_index, y, x1, x2))
            if 'Description' in url_key:
                description_index = cluster_index
            if 'Amount' in url_key:
                amount_index = cluster_index

    if cluster_index == 0:
        max_cluster_headers = cluster_headers
        
    if amount_index is not None:
        amount_cluster_headers = cluster_headers

    if description_index is not None:
        break
    else:
        cluster_index += 1
        cluster_headers = []
        
if description_index is None:
    if amount_index is not None:
        cluster_headers = amount_cluster_headers
    else:
        cluster_headers = max_cluster_headers
        
main_table_headers = cluster_headers

print(json.dumps(main_table_headers, indent=2))

[(1122, 6), (1176, 4), (2514, 2), (2867, 2)]
[
  [
    "http://bizlem.io/PurchaseOrderProcessing#CGST",
    20,
    1127,
    1771,
    1841
  ],
  [
    "http://bizlem.io/PurchaseOrderProcessing#SGST",
    20,
    1127,
    2032,
    2102
  ],
  [
    "http://bizlem.io/PurchaseOrderProcessing#DescriptionOfServices",
    21,
    1141,
    347,
    504
  ],
  [
    "http://bizlem.io/PurchaseOrderProcessing#DescriptionOfServices",
    22,
    1152,
    354,
    506
  ],
  [
    "http://bizlem.io/PurchaseOrderProcessing#Quantity",
    22,
    1152,
    1059,
    1099
  ],
  [
    "http://bizlem.io/PurchaseOrderProcessing#Unit",
    22,
    1152,
    1135,
    1320
  ]
]


In [ ]:
print(json.dumps(slist[36], indent=2))

In [11]:
header_line_index = list(set([ line_index for (_, line_index, _, _, _) in main_table_headers ]))[:4]
header_scan_index = [ len(slist[line_index]['words']) - 1 for line_index in header_line_index ]
header_lines = []
header_boxes = []

while True:
    right_most_line = None
    right_x = 0
    for i, line_index in enumerate(header_line_index):
        if header_scan_index[i] < 0: continue
        words = slist[line_index]['words']
        if words[header_scan_index[i]]['x2'] > right_x:
            right_x = words[header_scan_index[i]]['x2']
            right_most_line = i
    
    print(header_scan_index, right_most_line)
    is_middle = True
    arr = [ ' ' ] * len(header_line_index)
    
    current_line_index = header_line_index[right_most_line]
    current_word_index = header_scan_index[right_most_line]
    current_word = slist[current_line_index]['words'][current_word_index]
    
    arr[right_most_line] = current_word['word']
    
    left_x = current_word['x1']
    right_x = current_word['x2']
    y = slist[current_line_index]['y']
    
    print('\t', current_word['word'], y, left_x, right_x)
    
    is_middle = True
    for i, line_index in enumerate(header_line_index):
        if i == right_most_line or header_scan_index[i] < 0: continue
        words = slist[line_index]['words']
        word_index = header_scan_index[i]
        
        if words[word_index]['x2'] > current_word['x1']:
            arr[i] = words[word_index]['word']
            header_scan_index[i] -= 1
            
            left_x = min(left_x, words[word_index]['x1'])
            right_x = max(right_x, words[word_index]['x2'])
            
            is_middle = False
    
    print('\tis_middle', is_middle)
    if is_middle:
        if current_word_index >= 1:
            prev_word = slist[current_line_index]['words'][current_word_index - 1]
            mid_word_x = (current_word['x2'] + prev_word['x1']) // 2
            
            for i, line_index in enumerate(header_line_index):
                if i >= right_most_line or header_scan_index[i] < 0: continue
                words = slist[line_index]['words']
                word_index = header_scan_index[i]
                
                if words[word_index]['x1'] <= mid_word_x and words[word_index]['x2'] >= mid_word_x:
                    arr[i] = words[word_index]['word']
                    left_x = min(left_x, words[word_index]['x1'])
                    right_x = max(right_x, words[word_index]['x2'])
    
    valid_arr = False
    for word in arr:
        if re.search('[a-zA-Z]+', word) is not None:
            valid_arr = True
            break
            
    if valid_arr:
        header_lines.append(arr)
        header_boxes.append((current_line_index, y, left_x, right_x))
    
    header_scan_index[right_most_line] -= 1
    
    print(arr, y, left_x, right_x)
    
    negative_header_scan_index = [ scan_index for scan_index in header_scan_index if scan_index < 0 ]
    if len(negative_header_scan_index) == len(header_scan_index):
        break
    
    print()

for header_arr in header_lines:
    query_param = dict([ ('H' + str(4 - i), (header_arr[-(i + 1)] if i < len(header_arr) else ' ')) for i in range(4) ])
    print('%30s   %s' % (header_arr, query_param))
    

[5, 2, 8] 2
	 Amt 1152 2321 2381
	is_middle True
[' ', ' ', 'Amt'] 1152 2321 2381

[5, 2, 7] 2
	 Total 1152 2232 2308
	is_middle True
[' ', ' ', 'Total'] 1152 2232 2308

[5, 2, 6] 0
	 SGST 1127 2032 2102
	is_middle True
['SGST', ' ', ' '] 1127 2032 2102

[4, 2, 6] 0
	 CGST 1127 1771 1841
	is_middle True
['CGST', ' ', ' '] 1127 1771 1841

[3, 2, 6] 0
	 Taxable 1127 1509 1610
	is_middle True
['Taxable', ' ', ' '] 1127 1509 1610

[2, 2, 6] 0
	 nical 1127 1382 1466
	is_middle False
['nical', ' ', 'Disc.'] 1127 1382 1466

[1, 2, 5] 2
	 Unit Rate 1152 1135 1320
	is_middle True
[' ', ' ', 'Unit Rate'] 1152 1135 1320

[1, 2, 4] 2
	 Qty 1152 1059 1099
	is_middle False
['PER', ' ', 'Qty'] 1152 985 1099

[0, 2, 3] 0
	 HSNI 1127 867 968
	is_middle False
['HSNI', ' ', 'can'] 1127 867 968

[-1, 2, 2] 2
	 Goods/Services 1152 558 784
	is_middle False
[' ', 'Goodata', 'Goods/Services'] 1152 558 784

[-1, 1, 1] 1
	 of 1141 523 551
	is_middle False
[' ', 'of', 'of'] 1141 523 551

[-1, 0, 0] 2
	 Descripti

In [12]:
def request_header(header_arr, header_index, res_header):
    try:
        query_param = dict([ ('H' + str(4 - i), (header_arr[-(i + 1)] if i < len(header_arr) else ' ')) for i in range(4) ])
        print(query_param)
        r = requests.post(FLATTEN_URL, data=json.dumps(query_param))
        
        r = r.json()
        if len(r['response']['docs']) > 0:
            phrase = r['response']['docs'][0]['url']
            res_header[header_index] = phrase
            return phrase
        else:
            res_header[header_index] = None
            return None
            
    except Exception as e:
        traceback.print_exc()
    
res_header = [ None ] * len(header_lines)
threads = [ threading.Thread(target=request_header, args=(header_arr, header_index, res_header)) for header_index, header_arr in enumerate(header_lines) ]
print(len(threads))

print('requesting...')
for thread in threads:
    thread.start()
    
for thread in threads:
    thread.join()
print('finish requesting...')
    
print(res_header)
print(header_boxes)

12
requesting...
{'H4': 'Amt', 'H3': ' ', 'H2': ' ', 'H1': ' '}
{'H4': 'Total', 'H3': ' ', 'H2': ' ', 'H1': ' '}
{'H4': ' ', 'H3': ' ', 'H2': 'SGST', 'H1': ' '}
{'H4': ' ', 'H3': ' ', 'H2': 'CGST', 'H1': ' '}
{'H4': ' ', 'H3': ' ', 'H2': 'Taxable', 'H1': ' '}
{'H4': 'Disc.', 'H3': ' ', 'H2': 'nical', 'H1': ' '}
{'H4': 'Unit Rate', 'H3': ' ', 'H2': ' ', 'H1': ' '}
{'H4': 'Qty', 'H3': ' ', 'H2': 'PER', 'H1': ' '}
{'H4': 'can', 'H3': ' ', 'H2': 'HSNI', 'H1': ' '}
{'H4': 'Goods/Services', 'H3': 'Goodata', 'H2': ' ', 'H1': ' '}
{'H4': 'of', 'H3': 'of', 'H2': ' ', 'H1': ' '}
{'H4': 'Description', 'H3': 'Description', 'H2': ' ', 'H1': ' '}
finish requesting...
[['http://bizlem.io/PurchaseOrderProcessing#AMOUNT'], ['http://bizlem.io/PurchaseOrderProcessing#TOTAL'], ['http://bizlem.io/PurchaseOrderProcessing#SGST'], ['http://bizlem.io/PurchaseOrderProcessing#CGST'], ['http://bizlem.io/PurchaseOrderProcessing#TaxableAmount'], ['nical  Disc.'], ['http://bizlem.io/PurchaseOrderProcessing#Unit'], [

In [13]:
main_res_header = [ arr for arr in res_header if arr is not None ]
main_header_boxes = [ header_boxes[i] for i in range(len(res_header)) if res_header[i] is not None ]
res_header_url = [ None ] * len(main_res_header)

print(main_res_header)
print(main_header_boxes)
print()

def request_chosen_header(main_res_header, res_header_url):
    try:
        header_query = dict([ ('column' + str(len(main_res_header) - i), harr[0]) for i, harr in enumerate(main_res_header) ])
        print(json.dumps(header_query, indent=2))
        r = requests.post(CHOOSE_URL, data=json.dumps(header_query))
#         print('hello', r.text)
        print('json', json.dumps(r.json(), indent=2))
        r = r.json()['response']['docs'][0]['RequiredColumns'][0]
        r = json.loads(r)
        print(json.dumps(r, indent=2))
        
        for col_index, col_url in r.items():
            res_index = int(col_index[6:])
            res_header_url[-res_index] = col_url

        return r
            
    except Exception as e:
        traceback.print_exc()
    
request_chosen_header(main_res_header, res_header_url)
res_header_url

[['http://bizlem.io/PurchaseOrderProcessing#AMOUNT'], ['http://bizlem.io/PurchaseOrderProcessing#TOTAL'], ['http://bizlem.io/PurchaseOrderProcessing#SGST'], ['http://bizlem.io/PurchaseOrderProcessing#CGST'], ['http://bizlem.io/PurchaseOrderProcessing#TaxableAmount'], ['nical  Disc.'], ['http://bizlem.io/PurchaseOrderProcessing#Unit'], ['Per'], ['HSNI  can'], ['Goodata Goods/Services'], ['http://bizlem.io/PurchaseOrderProcessing#DescriptionOfServices'], ['http://bizlem.io/PurchaseOrderProcessing#DescriptionOfServices']]
[(22, 1152, 2321, 2381), (22, 1152, 2232, 2308), (20, 1127, 2032, 2102), (20, 1127, 1771, 1841), (20, 1127, 1509, 1610), (20, 1127, 1382, 1466), (22, 1152, 1135, 1320), (22, 1152, 985, 1099), (20, 1127, 867, 968), (22, 1152, 558, 784), (21, 1141, 523, 551), (22, 1152, 347, 506)]

{
  "column12": "http://bizlem.io/PurchaseOrderProcessing#AMOUNT",
  "column11": "http://bizlem.io/PurchaseOrderProcessing#TOTAL",
  "column10": "http://bizlem.io/PurchaseOrderProcessing#SGST",


['http://bizlem.io/PurchaseOrderProcessing#AMOUNT',
 'http://bizlem.io/PurchaseOrderProcessing#TOTAL',
 'http://bizlem.io/PurchaseOrderProcessing#SGST',
 'http://bizlem.io/PurchaseOrderProcessing#CGST',
 'http://bizlem.io/PurchaseOrderProcessing#TaxableAmount',
 'nical  Disc.',
 'http://bizlem.io/PurchaseOrderProcessing#Unit',
 'Per',
 'HSNI  can',
 'Goodata Goods/Services',
 'http://bizlem.io/PurchaseOrderProcessing#DescriptionOfServices',
 'http://bizlem.io/PurchaseOrderProcessing#DescriptionOfServices']

In [14]:
for i, header in enumerate(res_header_url):
    print(i, header, main_header_boxes[i])
    
print(len(main_header_boxes), len(res_header_url), len(main_res_header))

0 http://bizlem.io/PurchaseOrderProcessing#AMOUNT (22, 1152, 2321, 2381)
1 http://bizlem.io/PurchaseOrderProcessing#TOTAL (22, 1152, 2232, 2308)
2 http://bizlem.io/PurchaseOrderProcessing#SGST (20, 1127, 2032, 2102)
3 http://bizlem.io/PurchaseOrderProcessing#CGST (20, 1127, 1771, 1841)
4 http://bizlem.io/PurchaseOrderProcessing#TaxableAmount (20, 1127, 1509, 1610)
5 nical  Disc. (20, 1127, 1382, 1466)
6 http://bizlem.io/PurchaseOrderProcessing#Unit (22, 1152, 1135, 1320)
7 Per (22, 1152, 985, 1099)
8 HSNI  can (20, 1127, 867, 968)
9 Goodata Goods/Services (22, 1152, 558, 784)
10 http://bizlem.io/PurchaseOrderProcessing#DescriptionOfServices (21, 1141, 523, 551)
11 http://bizlem.io/PurchaseOrderProcessing#DescriptionOfServices (22, 1152, 347, 506)
12 12 12


In [15]:
header_pos = []
start_line_index = 0

for i, (line_index, y, left_x, right_x) in enumerate(main_header_boxes):
    if res_header_url[i] is None: continue
    header = res_header_url[i]
    header_pos.append((header, line_index, y, [left_x, right_x]))
    if start_line_index < line_index:
        start_line_index = line_index
    
header_pos.sort(key=lambda x: x[-1][0])

stop_line_index = start_line_index + 1

end = False
for line_index, words in enumerate(slist):
    if line_index <= stop_line_index or len(sres[line_index]['words']) == 0: 
        continue
        
    for word_index, word in enumerate(words['words']):
        url = sres_words[line_index]['words'][word_index]
        if SELECTED_LABEL not in url: continue
            
        url_key = url.split('#')[-1]
        if url_key == 'GrossTotal' or 'SGST' in url_key or 'CGST' in url_key or 'IGST' in url_key: 
            stop_line_index = line_index
            end = True
            break
    if end:
        break

# print(json.dumps(header_pos, indent=2))
print(start_line_index, stop_line_index)
print(header_pos)

22 45
[('http://bizlem.io/PurchaseOrderProcessing#DescriptionOfServices', 22, 1152, [347, 506]), ('http://bizlem.io/PurchaseOrderProcessing#DescriptionOfServices', 21, 1141, [523, 551]), ('Goodata Goods/Services', 22, 1152, [558, 784]), ('HSNI  can', 20, 1127, [867, 968]), ('Per', 22, 1152, [985, 1099]), ('http://bizlem.io/PurchaseOrderProcessing#Unit', 22, 1152, [1135, 1320]), ('nical  Disc.', 20, 1127, [1382, 1466]), ('http://bizlem.io/PurchaseOrderProcessing#TaxableAmount', 20, 1127, [1509, 1610]), ('http://bizlem.io/PurchaseOrderProcessing#CGST', 20, 1127, [1771, 1841]), ('http://bizlem.io/PurchaseOrderProcessing#SGST', 20, 1127, [2032, 2102]), ('http://bizlem.io/PurchaseOrderProcessing#TOTAL', 22, 1152, [2232, 2308]), ('http://bizlem.io/PurchaseOrderProcessing#AMOUNT', 22, 1152, [2321, 2381])]


In [23]:
items_all = []

def request_overlap(word, header_data, container):
    try:
        header, header_line_index, header_y, header_pos = header_data
        r = requests.post(OVERLAP_URL, data={
            'user_name': 'carrotrule_xyz.com',
            'project_name': 'DataExtractionPO-2',
            'Rule_Engine': 'RuleData-2',
            'RawJson': {
                'wX1': word['x1'],
                'wX2': word['x2'],
                'hX1': header_pos[0],
                'hX2': header_pos[1],
            },
        })
        
        r = r.json()
        
        container.append({
            'header': header if r['OVERLAP'] else '',
            'data': word['word'],
            'data_type': 'number' if is_float(word['word']) else 'string',
            'header_x1': header_pos[0],
            'header_x2': header_pos[1],
            'header_y': header_y,
            'data_x1': word['x1'],
            'data_x2': word['x2'],
            'data_y': word['y'],
        })
        
    except Exception as e:
        traceback.print_exc()

table_data = []
threads = []
for words_line, words in enumerate(slist[start_line_index + 1:stop_line_index]):
    for word in words['words']:
        for header_data in header_pos[::-1]:
            header, _, _, pos = header_data
            if x1 >= pos[0]:
                current_word = {
                    'x1': word['x1'],
                    'x2': word['x2'],
                    'y': words['y'],
                    'word': word['word'],
                }
                
                threads.append(threading.Thread(
                    target=request_overlap,
                    args=(current_word, header_data, table_data)
                ))
                
for thread in threads:
    thread.start()
    
for thread in threads:
    thread.join()

Traceback (most recent call last):
  File "<ipython-input-23-bb76186ee93c>", line 18, in request_overlap
    r = r.json()
  File "/usr/lib/python3.7/site-packages/requests/models.py", line 889, in json
    self.content.decode(encoding), **kwargs
  File "/usr/lib/python3.7/json/__init__.py", line 348, in loads
    return _default_decoder.decode(s)
  File "/usr/lib/python3.7/json/decoder.py", line 337, in decode
    obj, end = self.raw_decode(s, idx=_w(s, 0).end())
  File "/usr/lib/python3.7/json/decoder.py", line 355, in raw_decode
    raise JSONDecodeError("Expecting value", s, err.value) from None
json.decoder.JSONDecodeError: Expecting value: line 1 column 1 (char 0)
Traceback (most recent call last):
  File "<ipython-input-23-bb76186ee93c>", line 18, in request_overlap
    r = r.json()
  File "/usr/lib/python3.7/site-packages/requests/models.py", line 889, in json
    self.content.decode(encoding), **kwargs
  File "/usr/lib/python3.7/json/__init__.py", line 348, in loads
    return 

Traceback (most recent call last):
  File "<ipython-input-23-bb76186ee93c>", line 18, in request_overlap
    r = r.json()
  File "/usr/lib/python3.7/site-packages/requests/models.py", line 889, in json
    self.content.decode(encoding), **kwargs
  File "/usr/lib/python3.7/json/__init__.py", line 348, in loads
    return _default_decoder.decode(s)
  File "/usr/lib/python3.7/json/decoder.py", line 337, in decode
    obj, end = self.raw_decode(s, idx=_w(s, 0).end())
  File "/usr/lib/python3.7/json/decoder.py", line 355, in raw_decode
    raise JSONDecodeError("Expecting value", s, err.value) from None
json.decoder.JSONDecodeError: Expecting value: line 1 column 1 (char 0)
Traceback (most recent call last):
  File "<ipython-input-23-bb76186ee93c>", line 18, in request_overlap
    r = r.json()
  File "/usr/lib/python3.7/site-packages/requests/models.py", line 889, in json
    self.content.decode(encoding), **kwargs
  File "/usr/lib/python3.7/json/__init__.py", line 348, in loads
    return 

Traceback (most recent call last):
  File "<ipython-input-23-bb76186ee93c>", line 18, in request_overlap
    r = r.json()
  File "/usr/lib/python3.7/site-packages/requests/models.py", line 889, in json
    self.content.decode(encoding), **kwargs
  File "/usr/lib/python3.7/json/__init__.py", line 348, in loads
    return _default_decoder.decode(s)
  File "/usr/lib/python3.7/json/decoder.py", line 337, in decode
    obj, end = self.raw_decode(s, idx=_w(s, 0).end())
  File "/usr/lib/python3.7/json/decoder.py", line 355, in raw_decode
    raise JSONDecodeError("Expecting value", s, err.value) from None
json.decoder.JSONDecodeError: Expecting value: line 1 column 1 (char 0)
Traceback (most recent call last):
  File "<ipython-input-23-bb76186ee93c>", line 18, in request_overlap
    r = r.json()
  File "/usr/lib/python3.7/site-packages/requests/models.py", line 889, in json
    self.content.decode(encoding), **kwargs
  File "/usr/lib/python3.7/json/__init__.py", line 348, in loads
    return 

Traceback (most recent call last):
  File "<ipython-input-23-bb76186ee93c>", line 18, in request_overlap
    r = r.json()
  File "/usr/lib/python3.7/site-packages/requests/models.py", line 889, in json
    self.content.decode(encoding), **kwargs
  File "/usr/lib/python3.7/json/__init__.py", line 348, in loads
    return _default_decoder.decode(s)
  File "/usr/lib/python3.7/json/decoder.py", line 337, in decode
    obj, end = self.raw_decode(s, idx=_w(s, 0).end())
  File "/usr/lib/python3.7/json/decoder.py", line 355, in raw_decode
    raise JSONDecodeError("Expecting value", s, err.value) from None
json.decoder.JSONDecodeError: Expecting value: line 1 column 1 (char 0)
Traceback (most recent call last):
  File "<ipython-input-23-bb76186ee93c>", line 18, in request_overlap
    r = r.json()
  File "/usr/lib/python3.7/site-packages/requests/models.py", line 889, in json
    self.content.decode(encoding), **kwargs
  File "/usr/lib/python3.7/json/__init__.py", line 348, in loads
    return 

Traceback (most recent call last):
Traceback (most recent call last):
  File "<ipython-input-23-bb76186ee93c>", line 18, in request_overlap
    r = r.json()
  File "/usr/lib/python3.7/site-packages/requests/models.py", line 889, in json
    self.content.decode(encoding), **kwargs
  File "/usr/lib/python3.7/json/__init__.py", line 348, in loads
    return _default_decoder.decode(s)
  File "/usr/lib/python3.7/json/decoder.py", line 337, in decode
    obj, end = self.raw_decode(s, idx=_w(s, 0).end())
  File "/usr/lib/python3.7/json/decoder.py", line 355, in raw_decode
    raise JSONDecodeError("Expecting value", s, err.value) from None
json.decoder.JSONDecodeError: Expecting value: line 1 column 1 (char 0)
  File "<ipython-input-23-bb76186ee93c>", line 18, in request_overlap
    r = r.json()
  File "/usr/lib/python3.7/site-packages/requests/models.py", line 889, in json
    self.content.decode(encoding), **kwargs
  File "/usr/lib/python3.7/json/__init__.py", line 348, in loads
    return 

Traceback (most recent call last):
  File "<ipython-input-23-bb76186ee93c>", line 18, in request_overlap
    r = r.json()
  File "/usr/lib/python3.7/site-packages/requests/models.py", line 889, in json
    self.content.decode(encoding), **kwargs
  File "/usr/lib/python3.7/json/__init__.py", line 348, in loads
    return _default_decoder.decode(s)
  File "/usr/lib/python3.7/json/decoder.py", line 337, in decode
    obj, end = self.raw_decode(s, idx=_w(s, 0).end())
  File "/usr/lib/python3.7/json/decoder.py", line 355, in raw_decode
    raise JSONDecodeError("Expecting value", s, err.value) from None
json.decoder.JSONDecodeError: Expecting value: line 1 column 1 (char 0)
Traceback (most recent call last):
  File "<ipython-input-23-bb76186ee93c>", line 18, in request_overlap
    r = r.json()
  File "/usr/lib/python3.7/site-packages/requests/models.py", line 889, in json
    self.content.decode(encoding), **kwargs
  File "/usr/lib/python3.7/json/__init__.py", line 348, in loads
    return 

In [24]:
footer_headers = {
    'GrossTotal': number_regex,
    'TOTAL': number_regex,
    'SGSTAmt': number_regex,
    'CGSTAmt': number_regex,
    'IGSTAmt': number_regex,
    'TotalAmt': number_regex,
    'CGSTAmount': number_regex,
    'SGSTAmount': number_regex,
    'IGSTAmount': number_regex,
    'CGST': percentage_regex,
    'SGST': percentage_regex,
    'IGST': percentage_regex,
}

footer_lines = []
for lindex, line in enumerate(slist[stop_line_index:]):
    line_index = lindex + stop_line_index
    item = {}
    for word_index, words in enumerate(line['words']):
        url = sres_words[line_index]['words'][word_index]
        if SELECTED_LABEL in url:dat
            term = url.split('#')[-1]
            if term in footer_headers:
                paragraph = concat(slist, line_index, word_index, n_lines=0)
                regex = footer_headers[term]
                search_p = re.findall(number_regex, paragraph)
                if len(search_p) > 0:
                    item[url] = [ to_float(i) for i in search_p ]
    
    if len(item.keys()) == 0:
        continue
        
    item['line_no'] = line_index
    footer_lines.append(item)
    

    

NameError: name 'number_regex' is not defined

In [25]:

table_output = {
    'header_table_only_url': items_url,
    'header_table_all': items_all,
    'footer_table_only_url': footer_lines,
}

print(json.dumps(table_output, indent=2))

NameError: name 'items_url' is not defined

In [32]:
def request_validation(items):
#     try:
        query_params = {
            'header_table': items,
            'footer_table': []
        }
        
        r = requests.post(VALIDATION_URL, data=json.dumps(query_param))
        
        r = r.text
        print(r)
            
#     except Exception as e:
#         print(items, 'error request_validation')
#         print(e)
    
request_validation(items)

NameError: name 'items' is not defined

In [ ]:
slist

# Regex for Invoice Number 

In [14]:
def concat_rows_and_cols(slist, line_index, word_index, n_lines=10):
    paragraph = []
    for word in slist[line_index]['words'][word_index:]:
        nword = word.copy()
        nword['y'] = slist[line_index]['y']
        nword['line_index'] = line_index
        paragraph.append(nword)
    
    current_word = slist[line_index]['words'][word_index]
    
    for lindex in range(line_index + 1, min(line_index + n_lines + 1, len(slist))):
        line = slist[lindex]
        for word in line['words']:
            if word['x1'] >= current_word['x1'] - 15 and word['x1'] <= current_word['x2']:
                nword = word.copy()
                nword['y'] = line['y']
                nword['line_index'] = lindex
                paragraph.append(nword)
    
    return paragraph

def request_regex_word(word, word_index, container):
    try:
        r = requests.post(REGEX_URL, data=word['word'])
        container[word_index] = r.json()
    except Exception as e:
        traceback.print_exc()

def request_regex_header(slist, line_index, word_index, word_url, paragraph, regex_res):
    container = [ None ] * len(paragraph)
    threads = [ threading.Thread(target=request_regex_word, args=(word, word_index, container)) for word_index, word in enumerate(paragraph) ]
    for thread in threads:
        thread.start()
    
    for thread in threads:
        thread.join()

    regex_res.setdefault(word_url, {
        'Regex': [],
        'Integer': [],
        'Date': [],
        'UKS': [],
    })
    
    current_word = slist[line_index]['words'][word_index]
    print(current_word)
    for rr_index, rr in enumerate(container):
        word = paragraph[rr_index]
        print(word)
        for item in rr:
            regex_res[word_url]['Regex'].append({
                'Regex_Type': item['Regex_Type'],
                'Regex_Value': item['value'],
                'Regex_Id': item['Regex_id'],
                'Regex_Length': item['length'],
                'Regex_Line_OffSet': word['line_index'] - line_index,
                'Regex_X1 OffSet': word['x1'] - current_word['x1'],
                'Regex_X1-X2 OffSet': word['x1'] - current_word['x2'],
                'Regex_Y OffSet': word['y'] - slist[line_index]['y'],
                'Regex_X2-X1 OffSet': word['x2'] - current_word['x1'],
                'Regex_X2-X2 OffSet': word['x2'] - current_word['x2'],
            })
            break
        break
            
    for word in paragraph:
        search_result = re.search('\d+', word['word'])
        if search_result is not None:
            regex_res[word_url]['Integer'].append({
                'Integer_Value': word['word'][search_result.start():search_result.end()],
                'Integer_Length': search_result.end() - search_result.start(),
                'Integer_Line_OffSet': word['line_index'] - line_index,
                'Integer_X1 OffSet': word['x1'] - current_word['x1'],
                'Integer_X1-X2 OffSet': word['x1'] - current_word['x2'],
                'Integer_Y OffSet': word['y'] - slist[line_index]['y'],
                'Integer_X2-X1 OffSet': word['x2'] - current_word['x1'],
                'Integer_X2-X2 OffSet': word['x2'] - current_word['x2'],
            })
            
        search_result_date = re.search(date_regex, word['word'])
        if search_result_date is not None:
            regex_res[word_url]['Date'].append({
                'Date_Value': word['word'][search_result_date.start():search_result_date.end()],
                'Date_Length': search_result_date.end() - search_result_date.start(),
                'Date_Line_OffSet': word['line_index'] - line_index,
                'Date_X1 OffSet': word['x1'] - current_word['x1'],
                'Date_X1-X2 OffSet': word['x1'] - current_word['x2'],
                'Date_Y OffSet': word['y'] - slist[line_index]['y'],
                'Date_X2-X1 OffSet': word['x2'] - current_word['x1'],
                'Date_X2-X2 OffSet': word['x2'] - current_word['x2'],
            })

paragraph = concat_rows_and_cols(slist, 9, 0)
print(slist[9])
print(slist[12])
# print(json.dumps(paragraph, indent=2))
# regex_res = {}
# request_regex_header(slist, 1, 1, sres_words[1]['words'][1], paragraph, regex_res)
# regex_res

{'y': 727, 'words': [{'word': 'Invoice No.', 'x1': 1230, 'x2': 1417}, {'word': 'Dated', 'x1': 1748, 'x2': 1845}]}
{'y': 829, 'words': [{'word': '196', 'x1': 88, 'x2': 142}, {'word': '/198', 'x1': 154, 'x2': 223}, {'word': ',', 'x1': 236, 'x2': 246}, {'word': 'SAMUEL', 'x1': 263, 'x2': 423}, {'word': 'STREET,', 'x1': 441, 'x2': 612}, {'word': 'MASJID', 'x1': 621, 'x2': 762}, {'word': '(WEST),', 'x1': 794, 'x2': 944}, {'word': 'Delivery', 'x1': 1231, 'x2': 1360}, {'word': 'Note', 'x1': 1372, 'x2': 1450}, {'word': 'Mode/Terms', 'x1': 1745, 'x2': 1962}, {'word': 'of', 'x1': 1972, 'x2': 2002}, {'word': 'Payment', 'x1': 2010, 'x2': 2159}]}


In [ ]:
sres_words[8]

In [14]:
regex_urls = [
    "http://bizlem.io/PurchaseOrderProcessing#Invoice_Number",
    "http://bizlem.io/PurchaseOrderProcessing#Invoice_Date",
    "http://bizlem.io/PurchaseOrderProcessing#PAN_Number",
    "http://bizlem.io/PurchaseOrderProcessing#Purchase_Order_Number",
    "http://bizlem.io/PurchaseOrderProcessing#TAN",
    "http://bizlem.io/PurchaseOrderProcessing#GSTIN",
    "http://bizlem.io/PurchaseOrderProcessing#TRNO",
    "http://bizlem.io/PurchaseOrderProcessing#TRDate",
    "http://bizlem.io/PurchaseOrderProcessing#CIN",
    "http://bizlem.io/PurchaseOrderProcessing#VAT",
    "http://bizlem.io/PurchaseOrderProcessing#PODate",
    "http://bizlem.io/PurchaseOrderProcessing#Date",
]



threads = []
regex_res = {}
for line_index, line in enumerate(slist):
#     print(line)
    for word_index, word in enumerate(line['words']):
#         print('\t', line_index, word_index)
        word_url = sres_words[line_index]['words'][word_index]
        if word_url in regex_urls:
            print(line_index, word_index, word_url)
            paragraph = concat_rows_and_cols(slist, line_index, word_index)
            
            threads.append(threading.Thread(
                target=request_regex_header, 
                args=(slist, line_index, word_index, word_url, paragraph, regex_res),
            ))
            
for thread in threads:
    thread.start()
    
for thread in threads:
    thread.join()
    
print(json.dumps(regex_res, indent=2))
            

1 1 http://bizlem.io/PurchaseOrderProcessing#Invoice_Number
4 0 http://bizlem.io/PurchaseOrderProcessing#PAN_Number
6 2 http://bizlem.io/PurchaseOrderProcessing#Invoice_Date
20 0 http://bizlem.io/PurchaseOrderProcessing#GSTIN
22 0 http://bizlem.io/PurchaseOrderProcessing#Date
22 6 http://bizlem.io/PurchaseOrderProcessing#Purchase_Order_Number
46 0 http://bizlem.io/PurchaseOrderProcessing#Date
[{'Regex_Type': 'DIGITS', 'Regex_id': 'dg_4', 'length': 4, 'value': '2019'}, {'Regex_Type': 'Document', 'Regex_id': 'DOC_43', 'length': 5, 'value': '01-02'}] {'word': '01-02-2019', 'x1': 2450, 'x2': 2571, 'y': 200, 'line_index': 6}
[] {'word': '/Unique', 'x1': 264, 'x2': 354, 'y': 482, 'line_index': 20}
[] {'word': ':', 'x1': 244, 'x2': 251, 'y': 186, 'line_index': 4}
[] {'word': '&', 'x1': 1975, 'x2': 1983, 'y': 522, 'line_index': 22}
[{'Regex_Type': 'DIGITS', 'Regex_id': 'dg_2', 'length': 2, 'value': '01'}, {'Regex_Type': 'Document', 'Regex_id': 'DOC_43', 'length': 5, 'value': '01-02'}] {'word':